In [ ]:
from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName("ETL").config(
        "spark.jars",
        "/home/nader-muhammad/.local/share/DBeaverData/drivers/maven/maven-central/com.mysql/mysql-connector-j-8.2.0.jar").getOrCreate()

In [ ]:
df = spark.read.csv(r"/home/nader-muhammad/Documents/Data Engineering/DataSets/Messy_Employment_India_Dataset.csv", header=True, inferSchema=True)
df.show()

In [ ]:
df.write.format("jdbc").option("url","jdbc:mysql://localhost:3306/ETL_DWH").option("dbtable","sales_RawData_notclean").option("user","dbeaver").option("password","Dbeaver@123")\
        .option("driver","com.mysql.cj.jdbc.Driver").mode("append").save()


### ETL_Processing

### Standardize_Name(Consistency)

In [ ]:
from pyspark.sql.functions import regexp_replace, lower, upper, initcap

In [ ]:
df.select("Status").distinct().show()

In [ ]:
df = df.withColumn("Status", initcap("Status"))

In [ ]:
df.select("Status").distinct().show()

In [ ]:
df.select("Age Group").distinct().show()

In [ ]:
df = df.withColumn("Age Group", regexp_replace("Age Group", "25_34", "25-34"))
df = df.withColumn("Age Group", regexp_replace("Age Group", "35 - 44", "35-44"))
df = df.withColumn("Age Group", regexp_replace("Age Group", "18 to 24", "18-24"))

In [ ]:
df.select("Age Group").distinct().show()

In [ ]:
df.select("Education").distinct().show()

In [ ]:
df = df.withColumn("Education", regexp_replace("Education", "Masters", "Master"))

df = df.withColumn("Education", regexp_replace("Education", "Bachelors", "Bachelor's"))

df = df.withColumn("Education", regexp_replace("Education", "Ph.D", "PhD"))

df = df.withColumn("Education", initcap("Education"))


In [ ]:
df.select("Education").distinct().show()

In [ ]:
df.select("Industry").distinct().show()

In [ ]:
df = df.withColumn("Industry", regexp_replace("Industry", "Tech ", "Technology"))

df = df.withColumn("Industry", regexp_replace("Industry", "Healthcare", "Health"))

In [ ]:
df.select("Industry").distinct().show()

In [ ]:
df.select("Location").distinct().show()

In [ ]:
df = df.withColumn("Location", initcap("Location"))

In [ ]:
df.select("Location").distinct().show()

In [ ]:
df.select("AI Risk").distinct().show()

In [ ]:
df = df.withColumn("AI Risk", regexp_replace("AI Risk", "moderate", "medium"))
df = df.withColumn("AI Risk", initcap("AI Risk"))

In [ ]:
df.select("AI Risk").distinct().show()

In [ ]:
df.select("Years of Experience").distinct().show()

In [ ]:
df.select("Monthly Salary (INR)").distinct().show()

In [ ]:
df.select("Date Recorded").distinct().show()

### Handling_Nulls_ETL(Completeness)

In [ ]:
from pyspark.sql.functions import *

In [ ]:
from pyspark.sql.functions import col, count, when

df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()


In [ ]:
df = df.fillna({"Status":"Unknown", "Age Group":"0", "Education":"No Info", "Industry":"No Data","AI Risk":"No Level"})

In [ ]:
mean_value = df.select(mean("Years of Experience")).collect()[0][0]

In [ ]:
df = df.na.fill({"Years of Experience":mean_value})

In [ ]:
mean_val = df.select(mean("Monthly Salary (INR)")).collect()[0][0]

In [ ]:
df = df.na.fill({"Monthly Salary (INR)":mean_val})

In [ ]:
from pyspark.sql.functions import col, count, when

df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()


### Convert_DataTypes_ETL(Vaildity)

In [ ]:
df.printSchema()

In [ ]:
df = df.withColumn("Date Recorded", to_date("Date Recorded", "%m%d%y"))

In [ ]:
df.printSchema()

In [ ]:
df.write.format("jdbc").option("user", "dbeaver")\
                       .option("password", "Dbeaver@123")\
                       .option("url", "jdbc:mysql://localhost:3306/ETL_DWH")\
                       .option("dbtable", "clean_sales")\
                       .option("driver", "com.mysql.cj.jdbc.Driver").mode("append").save()